In [2]:
# !pip install lbl2vec
from lbl2vec import Lbl2Vec

In [ ]:
# !python -m pip install --upgrade pip

In [ ]:
# !pip install wheel
# !python -m pip install -U pip setuptools

In [1]:
# !pip install pip==22.1.2
# !pip list

In [5]:
# !pip install fasttext

In [6]:
# import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
# import gensim.models as g

# # model="../../../../enwiki_dbow/doc2vec.bin"  #point to downloaded pre-trained doc2vec model
# model='../../../../cc.en.300.bin'
# #load model
# m = g.Doc2Vec.load(model)

# Keywords

In [44]:
conflict = ['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army']
politics = ['politics', 'debate', 'election', 'government', 'democracy']
fashion = ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit']
culture = ['culture', 'opera', "festival", 'collection', 'exhibition', 'art', 'museum']
sports = ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play']
education = ['education', 'professor', 'study', 'research', 'university']
# football = []
# other

descriptive_keywords = [conflict, politics, fashion, culture, sports, education]
# How to avoid words with high tf?

In [38]:
descriptive_keywords

[['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army'],
 ['politics', 'debate', 'election', 'government', 'democracy'],
 ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit'],
 ['opera', 'festival', 'collection', 'exhibition', 'art', 'museum'],
 ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play'],
 ['education', 'professor', 'study', 'research', 'university']]

In [9]:
import os
import pickle

from collections import Counter

In [12]:
WORDLIST_LOCATION = '../../../../data/enwiki_city_pairs_lemmatised/NOUN_SPACY_LARGE_REAL_WORDS'
wordlists = []
city_pairs = []

for root, dirs, files in os.walk(WORDLIST_LOCATION, topdown=True):
    for name in files:
        file_path = os.path.join(root, name)
        
        with open(file_path, 'rb') as fp:
            wordlists.append(pickle.load(fp))
            city_pairs.append(name.split('__')[1])

In [17]:
# Sort by number of words

data = list(zip(wordlists, city_pairs))
sorted_data = sorted(data, key=lambda x: len(x[0]), reverse=True)

for x in sorted_data[:20]:
    print(len(x[0]), x[1])


427850 paris_london
215618 london_manchester
156600 london_birmingham
152094 london_berlin
151175 paris_berlin
129050 paris_rome
119845 london_glasgow
105319 berlin_vienna
101090 paris_vienna
97205 madrid_barcelona
94912 berlin_munich
90804 london_rome
81853 london_vienna
81411 berlin_hamburg
80532 paris_brussels
75608 birmingham_manchester
71275 milan_rome
68454 rome_naples
64397 paris_lyon
63562 berlin_frankfurt


In [76]:
len(sorted_data[0][0]), sorted_data[0][1]
c = Counter(sorted_data[0][0])
most_common = c.most_common()

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm

In [31]:
GLOVE_LOCATION = '../../../../glove.42B.300d.txt'

In [32]:
embeddings_dict = {}

In [33]:
num_lines = sum(1 for line in open(GLOVE_LOCATION,'r', encoding="utf-8"))

with open(GLOVE_LOCATION, 'r', encoding="utf-8") as f:
     for line in tqdm(f, total=num_lines):
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

  0%|          | 0/1917495 [00:00<?, ?it/s]

In [ ]:
descriptive_keywords

In [ ]:
# Calculate distance between two words

In [34]:
from scipy.spatial.distance import cosine

word_1 = embeddings_dict['fashion']
word_2 = embeddings_dict['war']
print(1 - cosine(word_1, word_2))

0.3141452968120575


In [145]:
#most_common

words = [x[0] for x in most_common[:1000]]
print(words)

['art', 'year', 'work', 'time', 'city', 'exhibition', 'world', 'opera', 'school', 'music', 'company', 'artist', 'gallery', 'film', 'member', 'museum', 'festival', 'war', 'society', 'painting', 'family', 'collection', 'tour', 'service', 'life', 'book', 'study', 'performance', 'concert', 'group', 'fashion', 'career', 'role', 'day', 'office', 'father', 'theatre', 'house', 'woman', 'government', 'country', 'century', 'son', 'place', 'production', 'studio', 'orchestra', 'director', 'college', 'number', 'man', 'album', 'history', 'event', 'design', 'child', 'week', 'month', 'death', 'ballet', 'age', 'end', 'friend', 'university', 'club', 'portrait', 'business', 'series', 'song', 'period', 'professor', 'stage', 'wife', 'painter', 'magazine', 'band', 'state', 'people', 'academy', 'summer', 'symphony', 'daughter', 'student', 'success', 'award', 'title', 'home', 'amp', 'mother', 'king', 'project', 'brother', 'street', 'model', 'line', 'flight', 'debut', 'record', 'law', 'degree', 'air', 'version

In [286]:
from pprint import pprint

In [390]:
%%time

verbose1, verbose2 = True, True
similar_categories = []
for word in words:
    closeness = []
    if verbose1 or verbose2:
        print('word:', word)
    for category in descriptive_keywords:
        summed_similarity = 0
        for keyword in category:
            keyword_vector = embeddings_dict[keyword]        
            word_vector = embeddings_dict[word]
            value = 1 - cosine(keyword_vector, word_vector)
            summed_similarity += value
        
        normalized_similarity = summed_similarity/len(category)
        closeness.append((normalized_similarity, category[0]))
        
        if verbose1:
            print('___________________________')
            print('===>', '\tcategory:', category[0], '\n\tsimilarity:', normalized_similarity, f"\n\tkeywords: {category}", )
        
    print()
    similar_category = max(closeness)
    
    if similar_category[0] > 0.25:
        similar_categories.append((word, similar_category))
    else:
        print('-->', (word, similar_category))
    
    if verbose2:
        #print('category similarity:')
        # pprint(sorted(closeness, key=lambda x: x[0], reverse=True))
        print()
        print(f"==> \tword: {word}, \n\tcategory:, {similar_category[1]}, \n\tvalue:, {similar_category[0]}")
        print('='*100)
        print('\n')

print('\nwords:', words, '\n')

similar_categories

word: football
___________________________
===> 	category: war 
	similarity: 0.29382860474288464 
	keywords: ['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army']
___________________________
===> 	category: politics 
	similarity: 0.30098319351673125 
	keywords: ['politics', 'debate', 'election', 'government', 'democracy']
___________________________
===> 	category: fashion 
	similarity: 0.2788689061999321 
	keywords: ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit']
___________________________
===> 	category: culture 
	similarity: 0.3087033288819449 
	keywords: ['culture', 'opera', 'festival', 'collection', 'exhibition', 'art', 'museum']
___________________________
===> 	category: sport 
	similarity: 0.5073593429156712 
	keywords: ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play']
___________________________
===> 	category: education 
	similarity: 0.33018014430999754 
	keywords: ['education', 'professor', 'study', 'research', 'university'

[('football', (0.5073593429156712, 'sport')),
 ('club', (0.5227627158164978, 'sport')),
 ('archery', (0.2930993586778641, 'sport')),
 ('athlete', (0.3517800590821675, 'sport')),
 ('ball', (0.4474520981311798, 'sport')),
 ('biking', (0.26202235583748135, 'sport')),
 ('student', (0.6046075105667115, 'education')),
 ('catwalk', (0.3622944504022598, 'fashion')),
 ('geek', (0.29754214733839035, 'fashion')),
 ('school', (0.5707644820213318, 'education')),
 ('helmet', (0.2644780533654349, 'sport')),
 ('poison', (0.3084911573678255, 'war')),
 ('sunglasses', (0.30030443022648495, 'fashion')),
 ('philosophy', (0.4873074173927307, 'education')),
 ('brain', (0.4037298858165741, 'education'))]

In [285]:
words = ["football", "club", "archery", "athlete", "ball", "biking", "student", 'catwalk', 'brainfreeze', 'nerd', 'geek', 
         'school', 'lawschool', 'helmet', 'poison', 'sunglasses', 'philosophy', 'brain']

In [320]:
similar_categories

[('football', (0.5073593429156712, 'sport')),
 ('club', (0.5227627158164978, 'sport')),
 ('archery', (0.2930993586778641, 'sport')),
 ('athlete', (0.3517800590821675, 'sport')),
 ('ball', (0.4474520981311798, 'sport')),
 ('biking', (0.26202235583748135, 'sport')),
 ('student', (0.6046075105667115, 'education')),
 ('catwalk', (0.3622944504022598, 'fashion')),
 ('geek', (0.29754214733839035, 'fashion')),
 ('school', (0.5707644820213318, 'education')),
 ('helmet', (0.2644780533654349, 'sport')),
 ('poison', (0.3084911573678255, 'war')),
 ('sunglasses', (0.30030443022648495, 'fashion')),
 ('philosophy', (0.4873074173927307, 'education')),
 ('brain', (0.4037298858165741, 'education'))]

In [291]:
categories_dict = {key[0]: 0 for key in descriptive_keywords}
categories_dict

{'war': 0,
 'politics': 0,
 'fashion': 0,
 'culture': 0,
 'sport': 0,
 'education': 0}

In [321]:
for x in similar_categories:
    categories_dict[x[1][1]] += x[1][0] #print(x[1])
    
categories_dict

# sorted(categories_dict, key=lambda )
results = list(sorted(categories_dict.items(), key=lambda item: item[1], reverse=True))
pprint(results)
print(f"\nThe dominant category is: '{results[0][0]}', followed by: '{results[1][0]}'.")

[('sport', 13.244769919131487),
 ('education', 10.33204647898674),
 ('fashion', 4.800705139835675),
 ('war', 1.5424557868391275),
 ('politics', 0),
 ('culture', 0)]

The dominant category is: 'sport', followed by: 'education'.


In [50]:
words

['art',
 'year',
 'work',
 'time',
 'city',
 'exhibition',
 'world',
 'opera',
 'school',
 'music',
 'company',
 'artist',
 'gallery',
 'film',
 'member',
 'museum',
 'festival',
 'war',
 'society',
 'painting']

In [69]:
import fasttext

ModuleNotFoundError: No module named 'fasttext'

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

vectors = load_vectors("../../../../cc.en.300.vec")

In [72]:
from gensim.models.wrappers import FastText

#model = FastText.load_fasttext_format('wiki.simple')

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [2]:
import gensim

In [3]:
path = "../../../../cc.en.300.vec"
model4 = gensim.models.fasttext.load_facebook_vectors(path)
# model3 = gensim.models.fasttext.load_facebook_model(path, encoding='utf-8')

ERROR! Session/line number was not unique in database. History logging moved to new session 70


NotImplementedError: Supervised fastText models are not supported

In [68]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors, Word2Vec

# wv_from_bin = KeyedVectors.load_word2vec_format(datapath("cc.en.300.bin"), binary=True)  # C bin forma
# word_vectors = KeyedVectors.load("../../../../cc.en.300.bin")
word_vectors2 = Word2Vec.load("../../../../cc.en.300.bin")
# wv_from_bin.save("cc_en_300.txt")

UnpicklingError: invalid load key, '\xba'.

In [60]:
from gensim.models import word2vec
model2 = word2vec.Word2Vec.load_word2vec_format("..//..//..//..//cc.en.300.bin", binary=True)
model2.save("cc_en_300.txt")

AttributeError: type object 'Word2Vec' has no attribute 'load_word2vec_format'

In [10]:
# init model #pretrained_w2v_model
model = Lbl2Vec(keywords_list=descriptive_keywords, tagged_documents=None, doc2vec_model=wv_from_bin)
# train model
model.fit()

NameError: name 'd' is not defined

In [ ]:
# get similarity scores for each new document from trained model
model.predict_new_docs(tagged_docs=tagged_docs)